In [1]:
! pip install autogen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 13.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.3/472.3 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.5 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.5/335.5 kB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [16]:
import os
from autogen import ConversableAgent


class DataOnboardingAgent:
    data_onboarding_system_message = "you are a helpful data processing agent"

    schema_inference_system_message = """please generate DDL based on input data
    here are the rules to follow
    * the DDL grammar follows ClickHouse style
    * the Table keyword MUST be replaced with Stream
    * all datatypes MUST be in lowercase, such uint32
    * all keywords MUST be in lowercase, such as nullable
    * all field names MUST keep same as in the json
    * composite types such as array, tuple, map cannot be nullable
    * should use composite types like array, map or tuple to represent complex structure in the json
    * if the data value is null, field type MUST be set as 'unknown'
    * return the result as a markdown sql code


    here is a sample of output DDL:
    ```sql
    CREATE STREAM car_live_data
    (
      `cid` string,
      `gas_percent` float64,
      `in_use` bool,
      `latitude` float64,
      `longitude` float64,
      `locked` bool,
      `speed_kmh` float64,
      `time` string,
      `total_km` float64
    )
    ```

    """

    schema_to_table_system_message = """based on generated DDL, please convert it into a json objec
    Rules:
    * for type string, it MUST be a single line for string
    
    for example, if the input DDL is:
    CREATE STREAM car_live_data
    (
      `cid` string,
      `gas_percent` float64,
      `in_use` bool,
      `composite` tuple(
          'x' int
          ),
    )
    
    the output of the json description of the DDL should be:
    ```json
    
    [
        {
            "name" : "cid", "type" : "string"
        },
        {
            "name" : "gas_percent", "type" : "float64"
        },
        {
            "name" : "in_use", "type" : "bool"
        },
        {
            "name" : "composite", "type" : "tuple('x' int)"
        }
    ]
    ```

    """

    field_summary_system_message = """ please generate a report to explain each fields of the schema,
    turn the hierachy into flatten when generating this report for each field,
    use '.' to connect the parents and children names
    output the result into a json object
    here is a sample of the output:
    [
        {
            "name": "eventversion",
            "type": "uint32",
            "description": "The version of the current event."
        },
        {
            "name": "open_24h",
            "type": "string",
            "description": "The price of the asset at the beginning of the last 24-hour period."
        }
    ]
    """

    analysis_recommendations_system_message = """ please propose what kind an analysis we can do based on input data and schema
    output into a json object which is an array
    """

    analysis_sql_generation_system_message = """ please generate 10 analysis SQL based on input schema and analysis recommendations
    output into a json object which is an array
    note, you need escape newlines if the output contains multiple lines string

    The generate SQL should follow these rules
    * the SQL follows the ClickHouse grammar
    * all method name MUST be in lower cases, following snake cases, for example : array_sum
    * no CROSS JOIN is supported

    As timeplus is a streaming processing platform, there are three different types of query regarding how to scan the data
    please randomly select one of these three patterns to generate SQL

    1 temperal window based analysis tumble window with 5 second window size
    following query return analysis result in a continously streaming query for every 5 second window
    select window_start, window_end, count(*) as count, max(c1) as max_c1
    from tumble(my_stream, 5s) group by window_start, window_end

    2 global aggregration which Global aggregation will start the aggregation for all incoming events since the query is submitted, and never ends.
    select count(*) as count, id as id
    from my_stream group by id

    3 historical aggreation, using table function, the query will just run traditional SQL that scan all historical data and return after query end
    select count(*) as count, id as id
    from table(my_stream) group by id


    #########
    here is a sample output:
    [
      {
        "sql": "select eventVersion, sum(videoSourceBandwidthBytesPerEvent + videoFecBandwidthBytesPerEvent + audioSourceBandwidthBytesPerEvent + audioFecBandwidthBytesPerEvent) as total_bandwidth_bytes from xray_stream group by eventVersion",
        "description": "Calculate the total bandwidth used per event version by summing up video, audio, and FEC bandwidths.",
        "name" : "Bandwidth Utilization Analysis"
      }
    ]

    """

    def __init__(self):
        self._llm_config = {
            "config_list": [
                {"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}
            ],
            "temperature": 0,
        }

        """
        self._llm_config = {
            "config_list": [
                {
                    "model": "gemma2:9b",
                    "base_url": "http://localhost:11434/v1",
                    "api_key": "ollama",
                }
            ]}
        """

        self.data_onboarding_agent = ConversableAgent(
            "data_onboarding_agent",
            system_message=self.data_onboarding_system_message,
            llm_config=self._llm_config,
            code_execution_config=False,
            max_consecutive_auto_reply=1,
            human_input_mode="NEVER",
        )

        self.schema_inference_agent = ConversableAgent(
            "schema_inference_agent",
            system_message=self.schema_inference_system_message,
            llm_config=self._llm_config,
            code_execution_config=False,
            max_consecutive_auto_reply=1,
            human_input_mode="NEVER",
        )

        self.schema_to_table_agent = ConversableAgent(
            "schema_to_table_agent",
            system_message=self.schema_to_table_system_message,
            llm_config=self._llm_config,
            code_execution_config=False,
            max_consecutive_auto_reply=1,
            human_input_mode="NEVER",
        )

        self.field_summary_agent = ConversableAgent(
            "field_summary_agent",
            system_message=self.field_summary_system_message,
            llm_config=self._llm_config,
            code_execution_config=False,
            max_consecutive_auto_reply=1,
            human_input_mode="NEVER",
        )

        self.analysis_recommendations_agent = ConversableAgent(
            "analysis_recommendations_agent",
            system_message=self.analysis_recommendations_system_message,
            llm_config=self._llm_config,
            code_execution_config=False,
            max_consecutive_auto_reply=1,
            human_input_mode="NEVER",
        )

        self.analysis_sql_generation_agent = ConversableAgent(
            "analysis_sql_generation_agent",
            system_message=self.analysis_sql_generation_system_message,
            llm_config=self._llm_config,
            code_execution_config=False,
            max_consecutive_auto_reply=1,
            human_input_mode="NEVER",
        )

    def process(self):
        message = (
            f"based on input data : {self.data}, and stream name : {self.stream_name}"
        )
        self.data_onboarding_agent.initiate_chats(
            [
                {
                    "recipient": self.schema_inference_agent,
                    "message": message,
                    "max_turns": 1,
                    "summary_method": "last_msg",
                },
                {
                    "recipient": self.field_summary_agent,
                    "message": "based on input DDL, add field summary",
                    "max_turns": 1,
                    "summary_method": "last_msg",
                },
                {
                    "recipient": self.analysis_recommendations_agent,
                    "message": f"based on input data : {self.data}, and field summary",
                    "max_turns": 1,
                    "summary_method": "last_msg",
                },
                {
                    "recipient": self.analysis_sql_generation_agent,
                    "message": "based on input schema and recommandations, generate analysis SQL, those metric seems like some video network package quality related data.",
                    "max_turns": 1,
                    "summary_method": "last_msg",
                },
            ]
        )

    def inference(self, data, stream_name):
        message = f"based on input data : {data}, and stream name : {stream_name}"
        self.data_onboarding_agent.initiate_chats(
            [
                {
                    "recipient": self.schema_inference_agent,
                    "message": message,
                    "max_turns": 1,
                    "summary_method": "last_msg",
                },
                {
                    "recipient": self.schema_to_table_agent,
                    "message": "please generate json expression for the schema",
                    "max_turns": 1,
                    "summary_method": "last_msg",
                }
            ]
        )

        return self.schema_inference_agent.last_message()["content"], self.schema_to_table_agent.last_message()["content"]

    def summary(self, data, columns):
        message = f"based on input data : {data}, and columns : {columns}"
        self.data_onboarding_agent.initiate_chats(
            [
                {
                    "recipient": self.field_summary_agent,
                    "message": message,
                    "max_turns": 1,
                    "summary_method": "last_msg",
                }
            ]
        )

        return self.field_summary_agent.last_message()["content"]

    def recommendations(self, data, columns, stream_name):
        message = f"based on input data : {data}, and columns : {columns} and stream name : {stream_name}"
        self.data_onboarding_agent.initiate_chats(
            [
                {
                    "recipient": self.analysis_sql_generation_agent,
                    "message": message,
                    "max_turns": 1,
                    "summary_method": "last_msg",
                }
            ]
        )

        return self.analysis_sql_generation_agent.last_message()["content"]


In [12]:
data_customer = '''
{
    "customer_id": 56,
    "name": "Emily Taylor",
    "email": "brianpeters@example.org",
    "phone": "522.227.0958x8414",
    "address": "231 Brown Orchard\nNorth Kristafurt, NC 05498",
    "home": null
}
'''
agent = DataOnboardingAgent()

ddl, ddl_json = agent.inference(data_customer, 'customer')

print(ddl)

print(ddl_json)


********************************************************************************
Starting a new chat....

********************************************************************************
data_onboarding_agent (to schema_inference_agent):

based on input data : 
{
    "customer_id": 56,
    "name": "Emily Taylor",
    "email": "brianpeters@example.org",
    "phone": "522.227.0958x8414",
    "address": "231 Brown Orchard
North Kristafurt, NC 05498",
    "home": null
}
, and stream name : customer

--------------------------------------------------------------------------------
schema_inference_agent (to data_onboarding_agent):

```sql
CREATE STREAM customer
(
  `customer_id` uint32,
  `name` string,
  `email` string,
  `phone` string,
  `address` string,
  `home` unknown
)
```

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

************************

In [5]:
data_credit_history = '''
{
    "history_id": 49,
    "customer_id": 49,
    "bank_name": "Montgomery Inc",
    "credit_score": 485,
    "outstanding_debt": 26706.08,
    "last_updated": 19674
}
'''
agent = DataOnboardingAgent()

agent.inference(data_credit_history, 'credit_history')


********************************************************************************
Starting a new chat....

********************************************************************************
data_onboarding_agent (to schema_inference_agent):

based on input data : 
{
    "history_id": 49,
    "customer_id": 49,
    "bank_name": "Montgomery Inc",
    "credit_score": 485,
    "outstanding_debt": 26706.08,
    "last_updated": 19674
}
, and stream name : credit_history

--------------------------------------------------------------------------------
schema_inference_agent (to data_onboarding_agent):

```sql
CREATE STREAM credit_history
(
  `history_id` uint32,
  `customer_id` uint32,
  `bank_name` string,
  `credit_score` uint32,
  `outstanding_debt` float64,
  `last_updated` uint32
)
```

--------------------------------------------------------------------------------


'```sql\nCREATE STREAM credit_history\n(\n  `history_id` uint32,\n  `customer_id` uint32,\n  `bank_name` string,\n  `credit_score` uint32,\n  `outstanding_debt` float64,\n  `last_updated` uint32\n)\n```'

In [17]:
data_unstructure = '''
{
	"_id": {
		"$oid": "67bc078e18939cdcb051732a"
	},
	"customer_id": 49,
	"raw_data": {
		"transaction_history": [
			{
				"date": {
					"$date": 1738368000000
				},
				"amount": 391.05
			},
			{
				"date": {
					"$date": 1738540800000
				},
				"amount": 923.33
			},
			{
				"date": {
					"$date": 1738972800000
				},
				"amount": 541.95
			},
			{
				"date": {
					"$date": 1736208000000
				},
				"amount": 116.17
			},
			{
				"date": {
					"$date": 1736899200000
				},
				"amount": 399.86
			},
			{
				"date": {
					"$date": 1739577600000
				},
				"amount": 625.11
			},
			{
				"date": {
					"$date": 1738195200000
				},
				"amount": 668.96
			},
			{
				"date": {
					"$date": 1736294400000
				},
				"amount": 472.46
			},
			{
				"date": {
					"$date": 1737072000000
				},
				"amount": 17.49
			}
		],
		"social_media_activity": {
			"platform": "Twitter",
			"activity_score": 34
		},
		"miscellaneous": {
			"notes": "Policy wish success begin candidate raise state.",
			"risk_flags": "Medium"
		}
	}
}
'''
agent = DataOnboardingAgent()

ddl, ddl_json = agent.inference(data_unstructure, 'unstructure')

print(ddl)

print(ddl_json)


********************************************************************************
Starting a new chat....

********************************************************************************
data_onboarding_agent (to schema_inference_agent):

based on input data : 
{
	"_id": {
		"$oid": "67bc078e18939cdcb051732a"
	},
	"customer_id": 49,
	"raw_data": {
		"transaction_history": [
			{
				"date": {
					"$date": 1738368000000
				},
				"amount": 391.05
			},
			{
				"date": {
					"$date": 1738540800000
				},
				"amount": 923.33
			},
			{
				"date": {
					"$date": 1738972800000
				},
				"amount": 541.95
			},
			{
				"date": {
					"$date": 1736208000000
				},
				"amount": 116.17
			},
			{
				"date": {
					"$date": 1736899200000
				},
				"amount": 399.86
			},
			{
				"date": {
					"$date": 1739577600000
				},
				"amount": 625.11
			},
			{
				"date": {
					"$date": 1738195200000
				},
				"amount": 668.96
			},
			{
				"date": {
					"$date": 1736294400000
				},
	